In [5]:
import socket

In [6]:
try:
    host_name = socket.gethostname()
    host_ip = socket.gethostbyname(host_name)
    print("Hostname :  ",host_name)
    print("IP : ",host_ip)
except:
    print("Unable to get Hostname and IP")

Hostname :   DESKTOP-NQAPMAS
IP :  192.168.56.1


In [8]:
HOST = host_ip  # The server's hostname or IP address
PORT = 55555        # The port used by the server

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((HOST, PORT))
    s.sendall(b'Hello, world')
    data = s.recv(1024)

print('Received', repr(data))

Received b'Hello, world'
